In [1]:
import tensorflow as tf


net = tf.keras.models.Sequential([
    tf.keras.layers.Dense(256, activation=tf.nn.relu),
    tf.keras.layers.Dense(10)
])

x = tf.random.uniform((2, 20))
net(x)

<tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[ 0.1715847 , -0.10307709,  0.08187982,  0.02779433,  0.2062802 ,
         0.02931514, -0.24573538,  0.13023314, -0.1421645 , -0.37283134],
       [ 0.10452854,  0.0217622 , -0.11679444, -0.02145112,  0.08768855,
        -0.12388965, -0.00372782,  0.3316537 ,  0.02154049, -0.06347504]],
      dtype=float32)>

In [2]:
class MLP(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.hidden = tf.keras.layers.Dense(units=256, activation=tf.nn.relu)
        self.out = tf.keras.layers.Dense(units=10)
        
    def call(self, x):
        return self.out(self.hidden((x)))

In [3]:
net = MLP()
net(x)

<tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[ 0.12421774,  0.1427388 ,  0.33290187,  0.10287586, -0.28874868,
        -0.05416413, -0.06458192, -0.1945131 , -0.09984571, -0.1162225 ],
       [ 0.21207519,  0.0181113 ,  0.20422581, -0.05967908, -0.43962288,
        -0.08328895,  0.3061719 ,  0.15206258,  0.02295022, -0.1021457 ]],
      dtype=float32)>

In [4]:
class MySequential(tf.keras.Model):
    def __init__(self, *args):
        super().__init__()
        self.modules = []
        for block in args:
            self.modules.append(block)
            
    def call(self, x):
        for module in self.modules:
            x = module(x)
        return x

In [5]:
net = MySequential(
    tf.keras.layers.Dense(units=256, activation=tf.nn.relu), 
    tf.keras.layers.Dense(10))
net(x)

<tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[ 0.22478274,  0.2067253 , -0.30728006,  0.23497938,  0.4144914 ,
        -0.04680262,  0.46753305,  0.13059507,  0.13277636, -0.2931348 ],
       [ 0.1583638 ,  0.24844168, -0.19212358,  0.05842307,  0.34166044,
        -0.08403128,  0.32861313, -0.12302326,  0.15209131, -0.20574093]],
      dtype=float32)>

In [6]:
class FixedHiddenMLP(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.flatten = tf.keras.layers.Flatten()
        self.rand_weight = tf.constant(tf.random.uniform((20, 20)))
        self.dense = tf.keras.layers.Dense(20, activation=tf.nn.relu)
        
    def call(self, inputs):
        x = self.flatten(inputs)
        x = tf.nn.relu(tf.matmul(x, self.rand_weight) + 1)
        x = self.dense(x)
        while tf.reduce_sum(tf.math.abs(x)) > 1:
            x /= 2
        return tf.reduce_sum(x)

In [7]:
net = FixedHiddenMLP()
net(x)

<tf.Tensor: shape=(), dtype=float32, numpy=0.88054085>

In [8]:
class NestMLP(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.net = tf.keras.Sequential()
        self.net.add(tf.keras.layers.Dense(64, activation=tf.nn.relu))
        self.net.add(tf.keras.layers.Dense(32, activation=tf.nn.relu))
        self.dense = tf.keras.layers.Dense(16, activation=tf.nn.relu)
        
    def call(self, inputs):
        return self.dense(self.net(inputs))
    
chimera = tf.keras.Sequential()
chimera.add(NestMLP())
chimera.add(tf.keras.layers.Dense(20))
chimera.add(FixedHiddenMLP())
chimera(x)

<tf.Tensor: shape=(), dtype=float32, numpy=0.7453592>